In [1]:
import random
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import trange
%matplotlib inline

train_text = './corpora/WarAndPeace.txt'
test_text = './corpora/AnnaKarenina.txt'

In [2]:
# предобработка данных
with open(train_text, 'r') as file:
    data_train = file.readlines()
data_train = ''.join(data_train)

with open(test_text, 'r') as file:
    data_test = file.readlines()
data_test = ''.join(data_test)

train_words = re.findall(r'\w+', data_train)
test_words = re.findall(r'\w+', data_test)

# берем только русские слова
alphabyte = ' абвгдежзийклмнопрстуфхцчшщъьыэюя'

data_train = []
for word in train_words:
    word = word.lower()
    flag = 0
    for char in word:
        if char not in alphabyte:
            flag = 1
            break
    if not flag:
        data_train += [word]

data_test = []
for word in test_words:
    word = word.lower()
    flag = 0
    for char in word:
        if char not in alphabyte:
            flag = 1
            break
    if not flag:
        data_test += [word]

data_train = ' '.join(data_train)
data_test = ' '.join(data_test)

# возьмем в качестве теста первые 1000 символов романа Анна Каренина, а остальную часть добавим в трейн
data_train += ' ' + data_test[1000:]
data_test = data_test[:1000]

In [3]:
# печатаем тестовые данные
print(data_test)

анна каренина один из самых знаменитых романов льва толстого начинается ставшей афоризмом фразой все счастливые семьи похожи друг на друга каждая несчастливая семья несчастлива по своему это книга о вечных ценностях о любви о вере о семье о человеческом достоинстве лев толстойроман широкого дыхания часть лев толстой анна каренина роман широкого дыхания анна каренина поразила современников вседневностью содержания необычайная свобода раскованность повествования удивительно сочетались в этом романе с цельностью художественного взгляда автора на жизнь он выступал здесь как художник и мыслитель и назначение искусства видел не в том чтобы неоспоримо разрешить вопрос а в том чтобы заставить любить жизнь в бесчисленных никогда не истощимых всех ее проявлениях в е годы один маститый писатель по видимому гончаров сказал достоевскому это вещь неслыханная это вещь первая кто у нас из писателей может поравняться с этим а в европе кто представит хоть что нибудь подобное ф м достоевский находил в но

## 1. Частотный метод по Шерлоку Холмсу

In [4]:
# подсчет частот символов в романе Война и Мир
frequency_dict_train = {}
for char in data_train:
    if char in frequency_dict_train:
        frequency_dict_train[char] += 1 / len(data_train)
    else:
        frequency_dict_train[char] = 1 / len(data_train)
# случайная перестановка символов
alphabyte = list(alphabyte)
alphabyte_new = alphabyte.copy()
random.shuffle(alphabyte_new)
dictionary_to_change = {i:j for i,j  in zip(alphabyte, alphabyte_new)}


# кодировка текста согласно перестановки
START = 0
LENGTH = 500
data_test_part = data_test[START:START + LENGTH]
data_test_encoded = []
for char in data_test_part:
    data_test_encoded += [dictionary_to_change[char]]
data_test_encoded = ''.join(data_test_encoded)
print('Исходная фраза:\n\n {}\n\n Закодированная фраза: \n\n {}\n'.format(data_test_part, data_test_encoded))

Исходная фраза:

 анна каренина один из самых знаменитых романов льва толстого начинается ставшей афоризмом фразой все счастливые семьи похожи друг на друга каждая несчастливая семья несчастлива по своему это книга о вечных ценностях о любви о вере о семье о человеческом достоинстве лев толстойроман широкого дыхания часть лев толстой анна каренина роман широкого дыхания анна каренина поразила современников вседневностью содержания необычайная свобода раскованность повествования удивительно сочетались в этом роман

 Закодированная фраза: 

 гннгзыгьбнлнгзяилнзлрзсгжоузрнгжбнлвоузьяжгняшзмюшгзвямсвяпязнгтлнгбвсщзсвгшабъзгхяьлржяжзхьгряъзшсбзстгсвмлшобзсбжюлзчяуя лзиьдпзнгзиьдпгзыг игщзнбстгсвмлшгщзсбжющзнбстгсвмлшгзчязсшябждзэвязынлпгзязшбтноузйбннясвщузязмцешлзязшбьбзязсбжюбзязтбмяшбтбсыяжзиясвялнсвшбзмбшзвямсвяъьяжгнзальяыяпязиоугнлщзтгсвюзмбшзвямсвяъзгннгзыгьбнлнгзьяжгнзальяыяпязиоугнлщзгннгзыгьбнлнгзчяьгрлмгзсяшьбжбннлыяшзшсбинбшнясвюцзсяибь гнлщзнбяеотгънгщзсшяеяигзьгсыяшгннясвюзчяш

In [5]:
# ищем по минимальному расстоянию по частотам в тесте и трейне
def find_relationship(dict_train, dict_test):
    key_in_use = set()
    decode_dict = {}
    for key_test, _ in sorted(dict_test.items(), key=lambda x: x[1], reverse=True):
        min_dist = np.inf
        min_key = None
        for key_train, _ in sorted(dict_train.items(), key=lambda x: x[1], reverse=True):
            if key_train not in key_in_use:
                cur_dist = abs(dict_train[key_train] - dict_test[key_test])
                if cur_dist < min_dist:
                    min_dist = cur_dist
                    min_key = key_train
        key_in_use.add(min_key)
        decode_dict[key_test] = min_key
    return decode_dict

In [6]:
# считаем частотность в тестовом отрывке
frequency_dict_test_encoded = {}
for char in data_test_encoded:
    if char in frequency_dict_test_encoded:
        frequency_dict_test_encoded[char] += 1 / len(data_test_encoded)
    else:
        frequency_dict_test_encoded[char] = 1 / len(data_test_encoded)

# сопоставляем с ближайшей с частотами на трейне
decode_dict = find_relationship(frequency_dict_train, frequency_dict_test_encoded)
# расшифровываем тестовый текст
data_test_decoded = ''
for char in data_test_encoded:
    data_test_decoded += decode_dict[char]
print('Расшифрованный текст: \n \n {}'.format(data_test_decoded))

Расшифрованный текст: 
 
 еаае пернатае оута тж иекзб жаекнатлзб рокеаос дгсе лодилочо аеятаенлиь илесюнй ещортжкок щрежой син ияеилдтсзн инкгт шобоэт урхч ае урхче пеэуеь анияеилдтсеь инкгь анияеилдтсе шо исонкх фло патче о сняазб ынааоильб о дъцст о снрн о инкгн о яндоснянипок уоилотаилсн днс лодилойрокеа ютропочо узбеать яеилг днс лодилой еаае пернатае рокеа ютропочо узбеать еаае пернатае шорежтде иосрнкнаатпос синуансаоилгъ иоунрэеать аноцзяейаеь исоцоуе реипосеааоилг шоснилсосеать хутстлндгао иоянледтиг с флок рокеа


## 2 Частотная модель на базе биграмм

In [7]:
# подсчет частот биграмм в романе Война и мир
frequency_dict_train = {}
for i in range(len(data_train)-1):
    bigram = data_train[i:i+2]
    if bigram in frequency_dict_train:
        frequency_dict_train[bigram] += 1
    else:
        frequency_dict_train[bigram] = 1

        
for key in frequency_dict_train:
    frequency_dict_train[key] = (frequency_dict_train[key] + 1) / (len(data_train) + 34 ** 2)
# Количество биграмм
len(frequency_dict_train.keys())

819

In [8]:
# считаем частотность биграмм в тестовом отрывке
frequency_dict_test_encoded = {}
for i in range(len(data_test_encoded)-1):
    bigram = data_train[i:i+2]
    if bigram in frequency_dict_test_encoded:
        frequency_dict_test_encoded[bigram] += 1 / len(data_test_encoded)
    else:
        frequency_dict_test_encoded[bigram] = 1 / len(data_test_encoded)


# сопоставляем по убыванию с частотами на трейне
decode_dict = find_relationship(frequency_dict_train, frequency_dict_test_encoded)
# расшифровываем тестовый текст
data_test_decoded_bigram = ''
for i in range(0, len(data_test_encoded), 2):
    bigram = data_train[i:i+2]
    data_test_decoded_bigram += decode_dict[bigram]
print('Расшифрованный текст: \n \n {}'.format(data_test_decoded_bigram))

Расшифрованный текст: 
 
  т уе о  кбы чбетьал п оонтья  яееняемседаол зелчаетол охо новож иовкаи шесвинстя ал пы  ии  ж чесит вдоия зт споевапоуд н кя соикв льа огарниид лужслак ивыа рулидннуе  пглресторгопоая нрое че нмо е втвал иа тон тыо  нмамуо о  а оо  нк  х о па аснеодсии  нихиви о заэту и  н т уи о  квостнелатьнай теергоныто стурый чеил о тй я ще л счие заовмеон смесьегжеулв льь еблуносянытиько о ы па к сто еутнониналал ьнатм  о тмисколказ умл вадоу о погдувбулал  п али стоимрои тнлыжнонывстаененескимтаеннн э па 


In [9]:
# сравниваем униграммы и биграммы
counter1 = 0
counter2 = 0
set_bigram = set()
for i, j, k in zip(data_test_decoded, data_test_decoded_bigram, data_test):
    if i == j:
        counter1 += 1
    if j == k:
        counter2 += 1

print("Доля отгаданных символов модели на униграммах: {} \
Доля отгаданных символов модели на биграммах: {}".format(counter1 / len(data_test_decoded), 
                                                         counter2 / len(data_test_decoded)))

Доля отгаданных символов модели на униграммах: 0.094 Доля отгаданных символов модели на биграммах: 0.082


__Вывод: униграммы работают с 33 + 1 символами, а биграммы с 34 * 34 биграммами, частотный метод лучше работает с униграммами  на небольшом корпусе данных, на больших лучше с биграммами__ 

## 3. Улучшение метода биграмм

_Идея: в цикле запускаем "умный" случайный поиск по типу Метрополиса-Хастинга:_

        - сэмплируем две буквы, будем их менять между собой
        - считаем новое значение правдоподобия входного текста (1000 первых букв Анны Каренины) по частотной статистике биграмм на трейне (Война и мир + Анна Каренина).
        - сравниваем отношение вероятностей получить данный текст до замены и после со случайным числом из диапазона [0,1). Если отношение больше, то принимаем замену, иначе нет.

_Будем запускать такой поиск много раз и выбирем оптимальную декодировку по критерию максимального правдоподобия._ 
_Как и в любых моделях жадного поиска это позволит получить разные локальные минимумы (строго, разный конечный результат)_

In [10]:
def decode_text(reverse_dict, text_encoded):
    text_decoded = ''
    for char in text_encoded:
        text_decoded += reverse_dict[char]
    return text_decoded

def calculate_log_prob(reverse_dict, text_encoded, freq_dict, n_gram=2):
    decoded_text = decode_text(reverse_dict, text_encoded)
    log_res = 0
    for ind in range(len(decoded_text) - 1):
        bigram = decoded_text[ind: ind + 2]
        bigram_value = freq_dict.get(bigram)
        if bigram_value is None:
            bigram_value = 1 / (len(decoded_text) + 33 ** 2)
        log_res += np.log(bigram_value)
    return log_res

In [11]:
NUM_ITERS = 100_000
NUM_TRIALS = 5
for j in range(NUM_TRIALS):

    ans = {key: key for key in alphabyte}
    cur_reverse = {j:i for i, j in ans.items()}

    prev_log_score = calculate_log_prob(cur_reverse, data_test_encoded, frequency_dict_train)
    if j == 0:
        list_of_keys = list(ans.keys())
        best_score = prev_log_score
        best_dict = ans.copy()
        best_reverse_dict = cur_reverse.copy()

    for i in trange(NUM_ITERS):

        key1 = random.choice(list_of_keys)
        key2 = random.choice(list_of_keys)
        ans_new = ans.copy()
        ans_new[key1], ans_new[key2] = ans_new[key2], ans_new[key1]
        
        cur_reverse = {j:i for i, j in ans_new.items()}
        new_log_score = calculate_log_prob(cur_reverse, data_test_encoded, frequency_dict_train)
        score = np.exp(new_log_score - prev_log_score)
    
        if score >= np.random.uniform(0, 1):
            prev_log_score = new_log_score
            ans = ans_new.copy()
                
        if new_log_score > best_score:
            best_score = new_log_score
            best_dict = ans.copy() 
            best_reverse_dict = cur_reverse.copy()
    print('trial number:{} best score: {}'.format(j, best_score))

100%|█████████████████████████████████| 100000/100000 [01:04<00:00, 1542.77it/s]


trial number:0 best score: -2803.436442337429


100%|█████████████████████████████████| 100000/100000 [01:05<00:00, 1518.67it/s]


trial number:1 best score: -2803.436442337429


100%|█████████████████████████████████| 100000/100000 [01:07<00:00, 1486.34it/s]


trial number:2 best score: -2803.436442337429


100%|█████████████████████████████████| 100000/100000 [01:07<00:00, 1478.13it/s]


trial number:3 best score: -2751.1571548629768


100%|█████████████████████████████████| 100000/100000 [01:07<00:00, 1475.38it/s]

trial number:4 best score: -2750.3832213710266


In [12]:
decode_text(best_reverse_dict, data_test_encoded)

'анна каденина омин из салых зналенитых доланов рьва торстого начинается ставшей афодизлол фдазой все счастривые сельи похоъи мдуг на мдуга каъмая несчастривая селья несчастрива по своелу это книга о вечных женностях о рюбви о веде о селье о черовеческол мостоинстве рев торстойдолан шидокого мыхания часть рев торстой анна каденина долан шидокого мыхания анна каденина подазира совделенников всемневностью сомедъания необычайная свобома даскованность повествования умивитерьно сочетарись в этол долан'

Вполне можно восстановить текст !!!!!

## 4. Расшифровка сообщений

In [13]:
message = '←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙\
↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴\
⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴\
↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏'

In [14]:
NUM_ITERS = 100_000
NUM_TRIALS = 5

uniques = list(set(message)) + [None] * (len(alphabyte) - len(list(set(message))))
for j in range(NUM_TRIALS):
    
    ans = {i:j for i,j in zip(alphabyte, uniques)}
    cur_reverse = {j:i for i, j in ans.items() if j is not None}

    prev_log_score = calculate_log_prob(cur_reverse, message, frequency_dict_train)
    if j == 0:
        list_of_keys = list(ans.keys())
        best_score = prev_log_score
        best_dict = ans.copy()
        best_reverse_dict = cur_reverse.copy()

    for i in trange(NUM_ITERS):

        key1 = random.choice(list_of_keys)
        key2 = random.choice(list_of_keys)
        ans_new = ans.copy()
        ans_new[key1], ans_new[key2] = ans_new[key2], ans_new[key1]
        
        cur_reverse = {j:i for i, j in ans_new.items()}
        new_log_score = calculate_log_prob(cur_reverse, message, frequency_dict_train)
        score = np.exp(new_log_score - prev_log_score)
    
        if score >= np.random.rand():
            prev_log_score = new_log_score
            ans = ans_new.copy()
                
        if new_log_score > best_score:
            best_score = new_log_score
            best_dict = ans.copy() 
            best_reverse_dict = cur_reverse.copy()
    print('trial number:{} best score: {}'.format(j, best_score))

100%|█████████████████████████████████| 100000/100000 [00:31<00:00, 3198.63it/s]


trial number:0 best score: -1360.1069707080533


100%|█████████████████████████████████| 100000/100000 [00:31<00:00, 3220.87it/s]


trial number:1 best score: -1360.1069707080533


100%|█████████████████████████████████| 100000/100000 [00:31<00:00, 3198.02it/s]


trial number:2 best score: -1238.4360269957488


100%|█████████████████████████████████| 100000/100000 [00:33<00:00, 2941.27it/s]


trial number:3 best score: -1238.4360269957488


100%|█████████████████████████████████| 100000/100000 [00:34<00:00, 2879.18it/s]

trial number:4 best score: -1238.4360269957488


In [15]:
decode_text(best_reverse_dict, message)

'если вы вимите нордальный или почти нордальный текст у этого соожшения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный жалл за послемнее четвертое замание курса ботя конечно я ничего не ожешах'

## 6 Применение схемы (бонус)

В максимально общем виде мы решили следующую задачу: имеется некоторая последовательность неизвестных по отдельности сущностей и их частота появления в других последовательностях, требуется идентификация сущностей в данной последовательности по их частоте появления. 

Это может быть полезно в задачах, где прямая идентификация затруднена или невозможна.
- по вычислительным или фундаментальным причинам (поиск элементарных частиц и осколков частиц в физике), сэмплирование в квантовой механике и квантовой химии (поиск конфигурации сложных молекул по спектру и минимуму энергий, кластерные образования молекул).
- биология: геномные последовательности, возможно под действием шума или опосредованными методами не дающими возможность идентифицировать аминокислоту и/или структуру по отдельности.
- расшифровка языков по сущностям (если народы жили в одной и той же среде обитания, то частоты использования слов для важнейших сущностей могут быть схожими). Для научных понятий это точно верно в силу универсальности науки (см. повесть "Универсальный язык"). Скорее это вспомогательный метод. Сейчас только древние и древнейшие языки требуют декодировку, а в них нет научных понятий. Возможно, это будет полезно, если человечество сможет преодолеть оковы капитализма и будет находить руины индустриальных цивилизаций в других галактиках.

Как и в методах сэмплирования можно использовать для решения задачи дискретной оптимизации, либо непрерывной оптимизации недифференцируемого функционала.